In [1]:
import pandas as pd

In [31]:
icnptso_cats = pd.read_csv("../data/icnptso.csv")
icnptso = pd.read_csv("../data/charities_active-icnptso.csv", index_col="org_id")
charities = pd.read_csv("../data/charities_active.csv", index_col="org_id")

icnptso_cats.loc[:, "code"] = icnptso_cats["Sub-group"].fillna(icnptso_cats["Group"]).fillna(icnptso_cats["Section"])
icnptso_cats = icnptso_cats.set_index("code")

charities = charities.join(
    icnptso.join(icnptso_cats, on="icnptso_code", how="left"),
    how="left",
)

charities.loc[:, "income_band"] = pd.cut(
    charities["income"],
    bins=[0, 10_000, 100_000, 1_000_000, 10_000_000, float("inf")],
    labels=["Under £10k", "£10k-£100k", "£100k-£1m", "£1m-£10m", "Over £10m"],
)

c:\users\drkan\code\ukcat\env\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1,8,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
charities.income_band.value_counts().sort_index()

Under £10k    65568
£10k-£100k    68889
£100k-£1m     33449
£1m-£10m       7683
Over £10m      1773
Name: income_band, dtype: int64

In [36]:
pd.crosstab(
    charities["income_band"],
    charities["source"],
)

source,ccew,ccni,oscr
income_band,,,
Under £10k,55272,1858,8438
£10k-£100k,58352,2107,8430
£100k-£1m,28111,1323,4015
£1m-£10m,6328,188,1167
Over £10m,1347,26,400


In [40]:
groups = charities.groupby(["income_band", "source"]).size()

In [52]:
pd.concat(
    [
        charities[(charities["income_band"] == index[0]) & (charities["source"] == index[1])].sample(20)
        for index, value in groups.iteritems()
    ]
).to_csv("../data/icnptso-to-test.csv")